In [6]:
import numpy as np
import pandas as pd
from yaml import safe_load
import os
from tqdm import tqdm

In [2]:
filenames = []
for file in os.listdir('F:/Data Science/Data Sets/T20 Match Score Prediction/t20s'):
    filenames.append(os.path.join('F:/Data Science/Data Sets/T20 Match Score Prediction/t20s',file))

In [3]:
filenames[:5]

['F:/Data Science/Data Sets/T20 Match Score Prediction/t20s\\1001349.yaml',
 'F:/Data Science/Data Sets/T20 Match Score Prediction/t20s\\1001351.yaml',
 'F:/Data Science/Data Sets/T20 Match Score Prediction/t20s\\1001353.yaml',
 'F:/Data Science/Data Sets/T20 Match Score Prediction/t20s\\1004729.yaml',
 'F:/Data Science/Data Sets/T20 Match Score Prediction/t20s\\1007655.yaml']

In [4]:
final_df = pd.DataFrame()
counter = 1
for file in tqdm(filenames):
    with open(file,'r') as f:
        df = pd.json_normalize(safe_load(f))
        df['match_id'] = counter
        final_df = final_df.append(df)
        counter+=1

100%|█████████████████████████████████████████████████████████████████████████████▉| 1432/1433 [15:16<00:00,  1.56it/s]


AttributeError: 'str' object has no attribute 'values'

In [5]:
final_df.head(2)

,innings,meta.data_version,meta.created,meta.revision,info.dates,info.gender,info.match_type,info.outcome.by.wickets,info.outcome.winner,info.overs,...,info.outcome.by.runs,info.match_type_number,info.neutral_venue,info.outcome.method,info.outcome.result,info.outcome.eliminator,info.supersubs.New Zealand,info.supersubs.South Africa,info.bowl_out,info.outcome.bowl_out
0,"[{'1st innings': {'team': 'Australia', 'delive...",0.9,2017-02-18,2,[2017-02-17],male,T20,5.0,Sri Lanka,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"[{'1st innings': {'team': 'Australia', 'delive...",0.9,2017-02-19,2,[2017-02-19],male,T20,2.0,Sri Lanka,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Checkpoint:
backup = final_df.copy()

In [7]:
backup.to_csv('F:/Data Science/Data Sets/T20 Match Score Prediction/t20_complete_data_backup.csv')

## Preprocessing:

In [29]:
# final_df = pd.read_csv('F:/Data Science/Data Sets/T20 Match Score Prediction/t20_complete_data_backup.csv')

In [20]:
final_df.shape

(1432, 28)

### Removing unwanted columns:

In [21]:
final_df.drop(columns=[
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.bowl_out',
    'info.outcome.bowl_out',
    'info.outcome.eliminator',
    'info.supersubs.New Zealand',
    'info.supersubs.South Africa',
    'info.outcome.by.runs',
    'info.match_type_number',
    'info.neutral_venue',
    'info.outcome.method',
    'info.outcome.result',
    'info.outcome.by.wickets'
],axis=1,inplace=True)

In [22]:
final_df.shape

(1432, 14)

### Handling 'info.gender' column:

In [23]:
final_df['info.gender'].value_counts()

male      966
female    466
Name: info.gender, dtype: int64

In [24]:
final_df = final_df[final_df['info.gender'] == "male"]
final_df.drop(columns=['info.gender'],axis=1,inplace=True)
final_df.shape

(966, 13)

### Handling 'info.match_type' and 'info.overs' column:

In [25]:
final_df['info.match_type'].value_counts()

T20    966
Name: info.match_type, dtype: int64

In [26]:
final_df['info.overs'].value_counts()

20    963
50      3
Name: info.overs, dtype: int64

In [27]:
final_df = final_df[final_df['info.overs'] == 20]
final_df.drop(columns=['info.overs','info.match_type'],axis=1,inplace=True)
final_df.shape

(963, 11)

In [28]:
final_df.head(2)

,innings,info.dates,info.outcome.winner,info.player_of_match,info.teams,info.toss.decision,info.toss.winner,info.umpires,info.venue,match_id,info.city
0,"[{'1st innings': {'team': 'Australia', 'delive...",[2017-02-17],Sri Lanka,[DAS Gunaratne],"[Australia, Sri Lanka]",field,Sri Lanka,"[MD Martell, P Wilson]",Melbourne Cricket Ground,1,NaN
0,"[{'1st innings': {'team': 'Australia', 'delive...",[2017-02-19],Sri Lanka,[DAS Gunaratne],"[Australia, Sri Lanka]",field,Sri Lanka,"[SD Fry, SJ Nogajski]","Simonds Stadium, South Geelong",2,Victoria


### Taking dataset backup  as pickle file:

In [30]:
import pickle
pickle.dump(final_df,open('F:/Data Science/Data Sets/T20 Match Score Prediction/dataset_level_1.pkl','wb'))

## Again loading the dataset:

In [1]:
import pickle
matches = pickle.load(open('F:/Data Science/Data Sets/T20 Match Score Prediction/dataset_level_1.pkl','rb'))
matches.head(2)

,innings,info.dates,info.outcome.winner,info.player_of_match,info.teams,info.toss.decision,info.toss.winner,info.umpires,info.venue,match_id,info.city
0,"[{'1st innings': {'team': 'Australia', 'delive...",[2017-02-17],Sri Lanka,[DAS Gunaratne],"[Australia, Sri Lanka]",field,Sri Lanka,"[MD Martell, P Wilson]",Melbourne Cricket Ground,1,NaN
0,"[{'1st innings': {'team': 'Australia', 'delive...",[2017-02-19],Sri Lanka,[DAS Gunaratne],"[Australia, Sri Lanka]",field,Sri Lanka,"[SD Fry, SJ Nogajski]","Simonds Stadium, South Geelong",2,Victoria


In [2]:
matches.iloc[0]['innings']

[{'1st innings': {'team': 'Australia',
   'deliveries': [{0.1: {'batsman': 'AJ Finch',
      'bowler': 'SL Malinga',
      'non_striker': 'M Klinger',
      'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
    {0.2: {'batsman': 'AJ Finch',
      'bowler': 'SL Malinga',
      'non_striker': 'M Klinger',
      'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
    {0.3: {'batsman': 'AJ Finch',
      'bowler': 'SL Malinga',
      'non_striker': 'M Klinger',
      'runs': {'batsman': 1, 'extras': 0, 'total': 1}}},
    {0.4: {'batsman': 'M Klinger',
      'bowler': 'SL Malinga',
      'non_striker': 'AJ Finch',
      'runs': {'batsman': 2, 'extras': 0, 'total': 2}}},
    {0.5: {'batsman': 'M Klinger',
      'bowler': 'SL Malinga',
      'non_striker': 'AJ Finch',
      'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
    {0.6: {'batsman': 'M Klinger',
      'bowler': 'SL Malinga',
      'non_striker': 'AJ Finch',
      'runs': {'batsman': 3, 'extras': 0, 'total': 3}}},
    {1.1: {'batsm

In [3]:
len(matches.iloc[0]['innings'])

2

In [4]:
matches.iloc[0]['innings'][0]['1st innings']['deliveries']

[{0.1: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.2: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.3: {'batsman': 'AJ Finch',
   'bowler': 'SL Malinga',
   'non_striker': 'M Klinger',
   'runs': {'batsman': 1, 'extras': 0, 'total': 1}}},
 {0.4: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 2, 'extras': 0, 'total': 2}}},
 {0.5: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 0, 'extras': 0, 'total': 0}}},
 {0.6: {'batsman': 'M Klinger',
   'bowler': 'SL Malinga',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 3, 'extras': 0, 'total': 3}}},
 {1.1: {'batsman': 'M Klinger',
   'bowler': 'KMDN Kulasekara',
   'non_striker': 'AJ Finch',
   'runs': {'batsman': 0, 'extras': 0, 'total': 

In [42]:
len(matches.iloc[0]['innings'][0]['1st innings']['deliveries'])

125

In [7]:
count = 1
delivery_df = pd.DataFrame()
for index, row in matches.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df = pd.DataFrame({
            'match_id':match_id,
            'teams':teams,
            'batting_team':batting_team,
            'ball':ball_of_match,
            'batsman':batsman,
            'bowler':bowler,
            'runs':runs,
            'player_dismissed':player_of_dismissed,
            'city':city,
            'venue':venue
        })
    delivery_df = delivery_df.append(loop_df)

In [8]:
delivery_df.head(2)

,match_id,teams,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue
0,2,"[Australia, Sri Lanka]",Australia,0.1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground
1,2,"[Australia, Sri Lanka]",Australia,0.2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground


In [9]:
delivery_df.shape

(115325, 10)

### Taking dataset backup  as pickle file:

In [10]:
import pickle
pickle.dump(delivery_df,open('F:/Data Science/Data Sets/T20 Match Score Prediction/dataset_level_2.pkl','wb'))

## Again loading the dataset:

In [44]:
import pickle
delivery_df = pickle.load(open('F:/Data Science/Data Sets/T20 Match Score Prediction/dataset_level_2.pkl','rb'))

### Our aim is to get following columns from 'delivery_df' dataframe:

- 1) batting_team : The team batting first.
- 2) bowling_team : The team bowling first.
- 3) current_scoe : What is the current score of batting team.
- 4) wickets_left : How many wickets are left for batting team.
- 5) crr : What is the current run rate of the batting team.
- 6) city : Where the match is being palyed.
- 7) balls_left : How may balls are left to complete the 20 overs.
- 8) last_five : What is the total runs from last five overs for batting team.

### 1) Creating 'batting_team' column:

- We already have 'batting_team' column, now using this we can create 'bowling_team' column.

In [45]:
def bowl(row):
    for team in row['teams']:
        if team != row['batting_team']:
            return team

In [46]:
delivery_df['bowling_team'] = delivery_df.apply(bowl,axis=1)

In [47]:
delivery_df.sample(2)

,match_id,teams,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
81,93,"[Pakistan, Scotland]",Pakistan,12.7,Sarfraz Ahmed,AC Evans,0,0,Edinburgh,"Grange Cricket Club, Raeburn Place",Scotland
81,597,"[Ireland, Netherlands]",Ireland,13.4,GC Wilson,MBS Jonkman,1,0,NaN,Dubai International Cricket Stadium,Netherlands


In [48]:
delivery_df.drop(columns=['teams'],inplace=True)

### 2) cleaning 'batting_team' and 'bowling_team':

In [49]:
delivery_df['batting_team'].unique()

array(['Australia', 'Hong Kong', 'Zimbabwe', 'India', 'Bangladesh',
       'New Zealand', 'South Africa', 'England', 'West Indies', 'Ireland',
       'Afghanistan', 'Pakistan', 'United Arab Emirates', 'Scotland',
       'Oman', 'Papua New Guinea', 'Sri Lanka', 'Netherlands', 'Nepal',
       'Vanuatu', 'Philippines', 'United States of America', 'Germany',
       'Ghana', 'Uganda', 'Kenya', 'Namibia', 'Nigeria', 'Botswana',
       'Guernsey', 'Denmark', 'Jersey', 'Italy', 'Norway', 'Thailand',
       'Malaysia', 'Maldives', 'Singapore', 'Kuwait', 'Bermuda', 'Canada',
       'Cayman Islands', 'Portugal', 'Gibraltar', 'Spain', 'Bhutan',
       'Qatar', 'Iran', 'Belgium', 'Isle of Man', 'Bulgaria', 'Romania'],
      dtype=object)

In [50]:
delivery_df['batting_team'].value_counts()

Pakistan                    10100
South Africa                 8434
India                        8257
New Zealand                  8232
Sri Lanka                    7937
West Indies                  7243
Australia                    6861
England                      6845
Afghanistan                  5206
Bangladesh                   4881
Ireland                      4539
Zimbabwe                     4361
Netherlands                  4011
United Arab Emirates         3069
Hong Kong                    2220
Scotland                     2214
Kenya                        1819
Oman                         1743
Malaysia                     1618
Nepal                        1588
Singapore                    1360
Papua New Guinea             1239
Canada                       1231
Namibia                       989
Jersey                        983
Thailand                      751
Nigeria                       739
Bermuda                       703
Cayman Islands                619
Vanuatu       

In [51]:

teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'    
]

In [52]:
delivery_df = delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(teams)]

In [53]:
delivery_df.sample(2)

,match_id,batting_team,ball,batsman,bowler,runs,player_dismissed,city,venue,bowling_team
117,467,Bangladesh,18.6,Abdur Razzak,JJ van der Wath,4,0,Cape Town,Newlands,South Africa
113,837,Sri Lanka,18.3,CK Kapugedera,Mohammad Irfan,6,0,Colombo,R Premadasa Stadium,Pakistan


In [54]:
delivery_df.shape

(63888, 10)

### 3) Dropping unwanted columns:

- We don't want 'batsman' & 'bowler' columns so we can remove them.

In [55]:
output = delivery_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]

In [56]:
output.head(2)

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground


### 4) Handling 'city' and 'venue' column:

In [57]:
pickle.dump(output,open('dataset_level2.pkl','wb'))

In [58]:
output.shape

(63888, 8)

In [59]:
output.isnull().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8548
venue                  0
dtype: int64

- Here, Out of (63,888) rows (8548) rows have null values for 'city' column.

In [60]:
output[output['city'].isnull()]['venue'].value_counts()

Dubai International Cricket Stadium        2969
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: venue, dtype: int64

- Here, we can use first word from 'venue' column as 'city' name where values are missing. 

In [61]:
np.where(output['city'].isnull(), output['venue'].str.split().apply(lambda x:x[0]), output['city'])

array(['Melbourne', 'Melbourne', 'Melbourne', ..., 'Colombo', 'Colombo',
       'Colombo'], dtype=object)

In [62]:
x = np.where(output['city'].isnull(), output['venue'].str.split().apply(lambda x:x[0]), output['city'])

In [63]:
output['city'] = x

<ipython-input-63-d2c7fa334a48>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['city'] = x


In [64]:
output.head(2)

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,Melbourne Cricket Ground
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,Melbourne Cricket Ground


In [65]:
output.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [66]:
output.drop(columns=['venue'],inplace=True)

C:\Users\LENOVO\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [67]:
output['city'].value_counts()

Colombo          4086
Mirpur           3420
Johannesburg     3331
Dubai            2969
Auckland         2532
                 ... 
Victoria          123
Dharamsala        122
Potchefstroom     122
Ahmedabad         121
Carrara            64
Name: city, Length: 86, dtype: int64

- Here, we are going to consider only those cities where minimum 5 matches (i.e. 600 balls) are palyed.

In [68]:
output['city'].value_counts()[output['city'].value_counts() > 600]

Colombo            4086
Mirpur             3420
Johannesburg       3331
Dubai              2969
Auckland           2532
Cape Town          2369
London             2106
Pallekele          2066
Barbados           2050
Sydney             1579
Melbourne          1574
Durban             1520
St Lucia           1487
Wellington         1350
Lauderhill         1238
Hamilton           1219
Centurion          1189
Manchester         1144
Abu Dhabi          1099
Mumbai              995
Nottingham          991
Southampton         987
Mount Maunganui     938
Chittagong          868
Kolkata             844
Lahore              748
Delhi               739
Nagpur              735
Chandigarh          627
Adelaide            623
Bangalore           620
St Kitts            618
Cardiff             615
Christchurch        614
Trinidad            611
Name: city, dtype: int64

In [69]:
eligible_cities = output['city'].value_counts()[output['city'].value_counts() > 600].index.tolist()

In [70]:
output = output[output['city'].isin(eligible_cities)]

In [71]:
output.shape

(50501, 7)

### 5) Creating total_score of each match for first innings:

In [72]:
total_df = output.groupby('match_id').sum()['runs'].reset_index()
output = output.merge(total_df,on='match_id')

In [73]:
output.head()

,match_id,batting_team,bowling_team,ball,runs_x,player_dismissed,city,runs_y
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,168
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,168
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,168
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,168
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,168


### 6) Creating 'current_score' column:

In [74]:
output['current_score'] = output.groupby('match_id').cumsum()['runs_x']

### 7) Creating 'balls_left' column:

In [75]:
output['over'] = output['ball'].apply(lambda x:str(x).split(".")[0])
output['ball_no'] = output['ball'].apply(lambda x:str(x).split(".")[1])

In [76]:
output['balls_bowled'] = (output['over'].astype('int')*6) + output['ball_no'].astype('int')

In [79]:
output['balls_left'] = 120 - output['balls_bowled']
output['balls_left'] = output['balls_left'].apply(lambda x:0 if x<0 else x)

In [80]:
output.drop(columns=['balls_bowled'],inplace=True)

### 8) Creating 'crr' column:

In [77]:
output['crr'] = round((output['current_score']*6)/output['balls_bowled'],2)

### 9) Creating 'wickets_left' column:

In [78]:
output['player_dismissed'] = output['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
output['player_dismissed'] = output['player_dismissed'].astype('int')
output['player_dismissed'] = output.groupby('match_id').cumsum()['player_dismissed']
output['wickets_left'] = 10 - output['player_dismissed']

### 10) Creating 'last_five' column:

In [83]:
groups = output.groupby('match_id')

In [84]:
match_ids = output['match_id'].unique()
last_five = []
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs_x'].values.tolist())

In [85]:
output['last_five'] = last_five

In [86]:
output.head(2)

,match_id,batting_team,bowling_team,ball,runs_x,player_dismissed,city,runs_y,current_score,over,ball_no,crr,wickets_left,balls_left,last_five
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,168,0,0,1,0.0,10,119,NaN
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,168,0,0,2,0.0,10,118,NaN


In [89]:
output.isnull().sum()

match_id                0
batting_team            0
bowling_team            0
ball                    0
runs_x                  0
player_dismissed        0
city                    0
runs_y                  0
current_score           0
over                    0
ball_no                 0
crr                     0
wickets_left            0
balls_left              0
last_five           12024
dtype: int64

In [96]:
index = output[output['last_five'].isnull()]['last_five'].index

In [101]:
index[:5]

Int64Index([0, 1, 2, 3, 4], dtype='int64')

In [100]:
output.shape[0]

50501

In [104]:
z = np.where(output['last_five'].isnull(), output['current_score'], output['last_five'])

In [105]:
output['last_five'] = z

In [106]:
output.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs_x              0
player_dismissed    0
city                0
runs_y              0
current_score       0
over                0
ball_no             0
crr                 0
wickets_left        0
balls_left          0
last_five           0
dtype: int64

In [110]:
output['target'] = output['runs_y'] + 1

### Backup:

In [111]:
import pickle
pickle.dump(output,open('F:/Data Science/Data Sets/T20 Match Score Prediction/dataset_level_3.pkl','wb'))

In [112]:
output.head(2)

,match_id,batting_team,bowling_team,ball,runs_x,player_dismissed,city,runs_y,current_score,over,ball_no,crr,wickets_left,balls_left,last_five,target
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,168,0,0,1,0.0,10,119,0.0,169
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,168,0,0,2,0.0,10,118,0.0,169


### Checkpoint:

In [113]:
final_df = output[['batting_team','bowling_team','current_score','wickets_left','crr','city', 'balls_left', 'last_five','target']]

In [116]:
final_df = final_df.sample(final_df.shape[0])

In [117]:
final_df.head(2)

,batting_team,bowling_team,current_score,wickets_left,crr,city,balls_left,last_five,target
45868,Pakistan,New Zealand,78,8,7.43,Auckland,57,36.0,172
39771,West Indies,Bangladesh,103,8,7.82,Mirpur,41,48.0,172


In [118]:
final_df.to_csv('F:/Data Science/Data Sets/T20 Match Score Prediction/t20_dataset_cleaned.csv')